In [297]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import bisect

torch.manual_seed(1337)

n_features = 1

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 33 # what is the maximum context length for predictions?
max_iters = 5000
epochs = 3
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
st_dims = 8
ac_dims = 1
rw_dims = 1
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab = 100

In [103]:
cols = ["Unnamed: 0", "X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "index"]
cols2 = ["Unnamed: 0", "observations", "next_observations", "index"]
obs_cols = ["X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "actions", "rewards", "rewardstg"]
dis_cols = [i + ' dis' for i in obs_cols]
extra_cols = ["Leg1", "Leg2", "actions", "rewards", "rewardstg"]
rwds_cols = ["rewards", "rewardstg"]
renames = {'Action':'actions', 'Reward':'rewards', 'observation_x':'observations', 'observation_y':'next_observations'}

In [180]:

def get_data():
    dataframe = pd.read_csv("medium_play.csv")
    dataframe['observation'] = list(dataframe.iloc[:, 1:9].values)
    observations = dataframe['observation'].copy()
    observations.drop(observations.index[0], inplace=True)
    dataframe.drop(dataframe.index[-1], inplace=True)
    observations = observations.reset_index()
    merged_data = dataframe.merge(observations, left_index=True, right_index=True)
    merged_data["terminals"] = False
    merged_data.loc[184753:, "Episode"] += 2000
    grouped_episodes = merged_data.groupby("Episode")
    merged_data = merged_data.rename(columns=renames)
    for episode, group in grouped_episodes:
        last_index = group.index[-1]  # Get the index of the last row in the group
        merged_data.loc[last_index, 'terminals'] = True
        every = group.index
        merged_data.loc[every, 'rewardstg'] = merged_data.loc[every, 'rewards'].cumsum()
        merged_data.loc[every, 'rewardstg'] -= merged_data.loc[every[-1], 'rewardstg']
        merged_data.loc[every, 'rewardstg'] *= -1
    observations = np.vstack(merged_data['observations'])
    next_observations = np.vstack(merged_data['next_observations'])
    actions = torch.tensor(merged_data["actions"].values, dtype=torch.long)
    rewards = torch.tensor(merged_data["rewardstg"].values, dtype=torch.float)
    observations = torch.tensor(observations, dtype=torch.float)
    next_observations = torch.tensor(next_observations)
    merged_data.drop(cols2, inplace=True, axis=1)
    return merged_data, actions, rewards, observations


data, actions, rewards, observations = get_data()



In [160]:
targets = pd.DataFrame()
decoder = []
total_vocab = 0
for i, col in enumerate(obs_cols):
    step = (data[col].max() - data[col].min())/(vocab-1)
    print(step)
    data[col+' dis'] = total_vocab + ( (data[col] - data[col].min()) / step).astype(int)
    targets[col+' dis'] = ( (data[col] - data[col].min()) / step).astype(int)
    decoder.append([step, data[col].min()])
    total_vocab += data[col + ' dis'].nunique()

0.020588951881485757
0.02400079429751694
0.044606553183661565
0.028204404946529503
0.07815593180030284
0.14864593563657819
0.010101010101010102
0.010101010101010102
0.030303030303030304
2.4190980318601105
8.904401554127759


In [260]:
#this class requres SAR sequence 
class UniformDiscretizer():
    def __init__(self, dataframe, vocab):
        self.columns = dataframe.columns
        self.mins = []
        self.maxs = []
        self.steps = []
        self.offsets = []
        self.data = dataframe
        self.vocab = vocab
        self.total_vocab = 0
        
    def fit_transform(self):
        discretized, targets = pd.DataFrame(), pd.DataFrame()
        t_vocab = 0
        
        for i, col in enumerate(self.columns):
            self.offsets.append(t_vocab)
            
            mini, maxi = self.data[col].min(), self.data[col].max()
            
            step = (maxi - mini)/(vocab-1)
            
            targets[col] = ((data[col] - mini) / step).astype(int)
            discretized[col] = self.offsets[-1] + targets[col]
            
            t_vocab += targets[col].nunique()
            
            self.mins.append(mini), self.maxs.append(maxi), self.steps.append(step)
            
        self.offsets.append(t_vocab)
        
        self.total_vocab = self.offsets[-1]
        
        return discretized, targets
    
    def trajectory_to_discretized(self, trajectory):
        return trajectory + offsets[:-1]
    
    def trajectory_to_target(self, trajectory):
        return trajectory - offsets[:-1]
    
    def token_to_discretized(self, token, column_idx):
        return token + self.offsets[column_idx]
        
    def token_to_target(self, token):
        column_idx = bisect.bisect_left(self.offsets, token) - 1
        return token - self.offsets[column_idx]
    
    def target_recover(self, token, column_idx):
        return (token+1)*self.steps[column_idx] + self.mins[column_idx]

In [269]:
data

,X Coordinate,Y Coordinate,Linear Velocity X,Linear Velocity Y,Angle,Angular Velocity,Leg1,Leg2,actions,rewards,Episode,terminals,rewardstg
0,0.009168,1.422375,0.466317,0.271966,-0.010169,-9.798397e-02,0.0,0.0,2.0,-3.524214,0,False,-203.813656
1,0.013870,1.427906,0.477506,0.245732,-0.017304,-1.427123e-01,0.0,0.0,3.0,-1.019767,0,False,-202.793890
2,0.018573,1.432837,0.477528,0.219063,-0.024437,-1.426711e-01,0.0,0.0,0.0,-0.046968,0,False,-202.746922
3,0.023168,1.438152,0.467235,0.236097,-0.032168,-1.546410e-01,0.0,0.0,2.0,-1.423334,0,False,-201.323588
4,0.027762,1.442868,0.467257,0.209417,-0.039898,-1.546060e-01,0.0,0.0,0.0,-0.106747,0,False,-201.216841
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400088,-0.087697,-0.000911,0.006983,0.000119,0.000588,-3.050588e-04,0.0,0.0,3.0,-0.650156,2499,False,10.126583
400089,-0.087571,-0.000910,0.012630,0.000002,0.000589,3.889892e-07,0.0,0.0,3.0,-0.581971,2499,False,10.708553
400090,-0.087386,-0.000911,0.018467,-0.000002,0.000588,-5.688639e-07,0.0,0.0,3.0,-0.595277,2499,False,11.303831
400091,-0.087375,-0.000908,0.001081,0.000118,0.000601,2.431244e-04,1.0,0.0,1.0,11.707871,2499,False,-0.404041


In [268]:
test_df = data[obs_cols]
UD = UniformDiscretizer(test_df, 100)
discretized, targets = UD.fit_transform()

In [263]:
for i in UD.steps:
    print(i)

0.020588951881485757
0.02400079429751694
0.044606553183661565
0.028204404946529503
0.07815593180030284
0.14864593563657819
0.010101010101010102
0.010101010101010102
0.030303030303030304
2.4190980318601105
8.904401554127759


In [264]:
UD.target_recover(54, 10)

-75.06643269867914

In [265]:
new_dataframe = data.iloc[:, 13:]

In [266]:
targets[:100]

,X Coordinate,Y Coordinate,Linear Velocity X,Linear Velocity Y,Angle,Angular Velocity,Leg1,Leg2,actions,rewards,rewardstg
0,49,76,60,87,49,51,0,0,66,39,40
1,50,77,60,86,49,50,0,0,99,40,40
2,50,77,60,85,49,50,0,0,0,41,40
3,50,77,60,85,49,50,0,0,66,40,40
4,50,77,60,84,49,50,0,0,0,41,40
...,...,...,...,...,...,...,...,...,...,...,...
95,83,37,73,31,42,53,0,0,0,41,54
96,84,36,74,32,42,53,0,0,66,40,54
97,85,35,75,31,42,52,0,0,99,40,54
98,85,33,75,30,42,52,0,0,99,40,54


In [267]:
data[:100]

,X Coordinate,Y Coordinate,Linear Velocity X,Linear Velocity Y,Angle,Angular Velocity,Leg1,Leg2,actions,rewards,Episode,terminals,rewardstg
0,0.009168,1.422375,0.466317,0.271966,-0.010169,-0.097984,0.0,0.0,2.0,-3.524214,0,False,-203.813656
1,0.013870,1.427906,0.477506,0.245732,-0.017304,-0.142712,0.0,0.0,3.0,-1.019767,0,False,-202.793890
2,0.018573,1.432837,0.477528,0.219063,-0.024437,-0.142671,0.0,0.0,0.0,-0.046968,0,False,-202.746922
3,0.023168,1.438152,0.467235,0.236097,-0.032168,-0.154641,0.0,0.0,2.0,-1.423334,0,False,-201.323588
4,0.027762,1.442868,0.467257,0.209417,-0.039898,-0.154606,0.0,0.0,0.0,-0.106747,0,False,-201.216841
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.709161,0.473466,1.042999,-1.281171,-0.611491,0.173463,0.0,0.0,0.0,-0.408761,0,False,-81.481813
96,0.720386,0.445115,1.114742,-1.256412,-0.602115,0.187515,0.0,0.0,2.0,-1.534894,0,False,-79.946919
97,0.731681,0.416103,1.123447,-1.286702,-0.594951,0.143288,0.0,0.0,3.0,-1.653984,0,False,-78.292934
98,0.743026,0.386443,1.129970,-1.316141,-0.589455,0.109919,0.0,0.0,3.0,-1.711878,0,False,-76.581056


In [149]:
new_dataframe.nunique()

X Coordinate dis         100
Y Coordinate dis         100
Linear Velocity X dis     98
Linear Velocity Y dis    100
Angle dis                 99
Angular Velocity dis      97
Leg1 dis                   2
Leg2 dis                   2
actions dis                4
rewards dis               74
rewardstg dis             99
dtype: int64

In [270]:
# data = torch.tensor(np.array(list(new_dataframe.values)).astype(int)).view(-1)
# targets = torch.tensor(np.array(list(targets.values)).astype(int)).view(-1)
print(data[0:12])
print(targets[0:12])

    X Coordinate  Y Coordinate  Linear Velocity X  Linear Velocity Y  \
0       0.009168      1.422375           0.466317           0.271966   
1       0.013870      1.427906           0.477506           0.245732   
2       0.018573      1.432837           0.477528           0.219063   
3       0.023168      1.438152           0.467235           0.236097   
4       0.027762      1.442868           0.467257           0.209417   
5       0.032419      1.446992           0.475033           0.183010   
6       0.037158      1.450511           0.485369           0.155961   
7       0.041897      1.453431           0.485402           0.129283   
8       0.046786      1.456764           0.499877           0.147573   
9       0.051672      1.460063           0.499658           0.145983   
10      0.056558      1.462764           0.499689           0.119304   
11      0.061360      1.464889           0.489080           0.093784   

       Angle  Angular Velocity  Leg1  Leg2  actions   rewards  

In [156]:
def get_batch():
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([targets[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    dt.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        x, y = get_batch()
        logits, loss = dt(x, targets=y)
        losses[k] = loss.item()
    out = losses.mean()
    dt.train()
    return out

In [157]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(3*block_size, 3*block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class TrajectoryTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.init_embd = nn.Embedding(total_vocab, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, ac_dims)

    def forward(self, sequence, targets=None):
        vocab_embedding = self.init_embd(sequence)#(B,T,C)
        B, T, C = vocab_embedding.shape
        pos_encoding = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = vocab_embedding + pos_encoding #(B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)

        preds = self.lm_head(acs) # (B,T,vocab)

        if targets is None:
            loss = None
        else:
            B, T, C = preds.shape
            preds = preds.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(preds, targets)

        return preds, loss

In [61]:
dt = DecisionTransformer().to(device)
ax, ay, rx, ry, sx, sy = get_intercalated()
embedding = dt.embeddings(sx, ax, rx)
print(embedding.shape)
preds, loss = dt.forward(sx, ax, rx, targets=ay)

torch.Size([16, 33, 64])


In [281]:
a = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0])

ind = np.argpartition(a, -4)[-4:]
print(a[ind])

[4 9 6 9]


In [299]:
top_pairs = [["a", "b"]]
# for i, b, c in zip(range(1), *top_pairs):
#     print(i, b, c)
a = [print(a, i) for a, i in enumerate(top_pairs)]

0 ['a', 'b']


In [ ]:
0

In [300]:
trajectory_length = st_dims + ac_dims + 2

In [298]:
def beam_search(model, context, st_width=3, ac_width=4, horizon=1, tr_width=1):
    window_by_index = [st_width]*st_dims + [ac_width]*ac_dims + [1] + [1]
    print(window_by_index)
    top_beams = [[context, 0, 0]]
    square_beams = []
    for i in range(trajectory_length):
        k = window_by_index[i]
        for beam in top_pairs:
            seq, prob = beam
            logits, __ = model(seq[-block_size:])
            logits = logits[-1, :]
            top_k = np.argpartition(logits, -k)[-k:]
            temp_beams = [[seq + UD.token_to_discretize(i), prob + logits[i], 0] for i in top_k]
            square_beams += temp_beams
            
        top_beams = sorted(my_list, key=lambda x: x[1])
        if i < st_dims:
            top_beams = top_beams[-k:]
        #probably incorperte i < st_dims + ac_dims toop_beams -st_width * ac_width
      
    top_rwds_idx = np.argsort([target_recover(x[0][-2], -2) + target_recover([0][-1], -1) for x in top_beams])
    return top_beams[top_rwds_idx[-tr_width:]]


[3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 1]


In [62]:
optimizer = torch.optim.AdamW(dt.parameters(), lr=learning_rate)

In [63]:
for _ in range(epochs):
    for iteration in range(max_iters):
      # every once in a while evaluate the loss on train and val sets
        if iteration % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iteration}: train loss {losses:.4f}")

        # sample a batch of data
        x, y = get_batch()

        # evaluate the loss
        logits, loss = dt(x, targets=y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

step 0: train loss 1.4075
step 100: train loss 1.0179


KeyboardInterrupt: 

In [12]:
import gymnasium as gym

In [52]:
def testing(model, desired_reward, iterations):
    env = gym.make("LunarLander-v2")
    copy_dr = desired_reward
    for i in range(iterations):
        # Reset it
        total_reward = 0.0
        desired_reward = copy_dr
        #numpy array (C)
        state, _ = env.reset(seed=None)

        # While the episode is not finished
        terminated = False
        time = 0

        rewards = torch.full((1, rw_dims), desired_reward, dtype=torch.float, device=device)
        states = torch.zeros((1, 1, st_dims), dtype=torch.float, device=device)
        states[0, 0, :] = torch.tensor(state, dtype=torch.float, device=device)
        actions = torch.zeros((1, 1), dtype=torch.long, device=device)
        actions[0, 0] = 0.0

        # define a trigger function, return True to start recording a new video:
        episode_trigger=lambda episode_id: (episode_id%1==0)

        while not terminated and time <= 500:
            input_st, input_ac, input_rw = states[:, -block_size:, :], \
                  actions[:, -block_size:], rewards[:, -block_size:]

            output, loss = dt(input_st, input_ac, input_rw)
            output = output[:, -1:, :].view(ac_dims)
            probabilities = F.softmax(output[:], dim=0)
            # ---> TODO: how to select an action
            # select action with the greatest probability according to policy
            action = int(torch.argmax(probabilities))
            # One step forward
            state, reward, terminated, _, _ = env.step(action)

            total_reward += reward
            desired_reward -= reward
            state = torch.tensor(state, dtype=torch.float, device=device).view(1, 1, st_dims)
            states = torch.cat((states, state), dim=1)

            reward = torch.tensor(desired_reward, dtype=torch.float, device=device).view(1, rw_dims)
            rewards = torch.cat((rewards, reward), dim=1)

            action = torch.tensor(action, dtype=torch.long, device=device).view(1, 1)
            actions = torch.cat((actions, action), dim=1)

            time += 1

      # Print reward
        print("total_reward = {}".format(total_reward))
    env.close()

In [53]:
testing(dt, 200, 10)

total_reward = 138.5807683382041
total_reward = 63.18579488790701
total_reward = 249.1230594821509
total_reward = 162.702864818653
total_reward = 127.74959205753673
total_reward = 131.1146253189614
total_reward = 236.75370396030746
total_reward = 29.350035516690298
total_reward = 229.6265528789939
total_reward = 143.16267473557133


In [ ]:
torch.save(dt, "first_iter.model")

# Load Model

In [37]:
dt = DecisionTransformer()
dt = torch.load("first_iter.model", map_location=torch.device('cpu'))
dt.eval()

DecisionTransformer(
  (st_embd): Linear(in_features=8, out_features=64, bias=True)
  (ac_embd): Embedding(4, 64)
  (rw_embd): Linear(in_features=1, out_features=64, bias=True)
  (position_embedding_table): Embedding(11, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0): Head(
            (key): Linear(in_features=64, out_features=16, bias=False)
            (query): Linear(in_features=64, out_features=16, bias=False)
            (value): Linear(in_features=64, out_features=16, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (1): Head(
            (key): Linear(in_features=64, out_features=16, bias=False)
            (query): Linear(in_features=64, out_features=16, bias=False)
            (value): Linear(in_features=64, out_features=16, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (2): Head(
            (key): Linear(in_features=64, out_f

In [39]:
testing(dt, 200)

total_reward = 156.74806023712523
total_reward = 178.84370687355053
total_reward = 140.25380092304758
total_reward = 231.18305450499892
total_reward = 139.46242194683865
total_reward = 227.9879113086424
total_reward = 129.69177142620666
total_reward = 135.01765524709964
total_reward = 159.87172585925848
total_reward = 223.41739922675993
